# Generate CSV files for weather DB creation
**Notebook is ETL operation which creates CSV files for 2018 weather data to be loaded in DB**

In [1]:
import pandas as pd

<br>
<br>

## Extended time weather data for 2018
* **WEATHER DATA FOR YEAR 2018 IS AVAIALABLE FOR TRAINING**
* **HOURELY WESTHER DATA IS AVAILABLE; WHICH NEEDS TO BE MERGED ON DUBLIN BUS DATA**
    * **Data can be merged using : DAYOFSERVICE, hour_PLANNEDTIME_ARR**
* **BUT, DUBLIN BUS DATA IS IN EXTENDED FORMAT, i.e. IF A BUS STARTS BEFORE 12 MIDNIGHT BUT ENDS ITS TRAVEL AFTER 12 MIDNIGHT, THEN IT IS RECORDED UNDER PRESENT DAY ONLY; HENCE ENTRIES WITH HOUR_PLANNEDTIME_ARR$\ >\ 23$ ARE ALSO FOUND**
    * **Extended time entries for arrival and departures time features are found where values are in excess of 86400 [24*3600]**
* **FROM DUBLIN BUS DATABASE, MAXIMUM VALUE OF EXTENDED HOURS IS $25$, WHICH IS $2$ HOURS INTO NEXT DAY**
<BR>
    
**As, weather data is in normal format, merging can be difficult. Hence, weather data entries for extended time are created**
$first\ 3\ entries\ for\ next\ day\ = 3\ Extended\ time\ entries\ for\ present\ day$

### Dataframe for 2018 Weather data
read weather data froppoing duplicate entries on feature dt : Result has only 1 entry for each hour of year 2018

In [2]:
df_weather = pd.read_excel("../DB/2018_historic_weather.xlsx",index_col=False).drop_duplicates(subset ="dt").reset_index(drop=True)

In [3]:
print("Shape of dataframe",df_weather.shape)
df_weather.head()

Shape of dataframe (8760, 25)


,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1514764800,2018-01-01 00:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.15,-6.49,3.84,5.79,...,240,NaN,NaN,NaN,NaN,40,520,Rain,light intensity shower rain,09n
1,1514768400,2018-01-01 01:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.14,-5.79,3.65,5.86,...,240,NaN,NaN,NaN,NaN,75,520,Rain,light intensity shower rain,09n
2,1514772000,2018-01-01 02:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.61,-5.77,3.85,5.99,...,240,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03n
3,1514775600,2018-01-01 03:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,4.64,-5.73,4.00,6.14,...,240,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03n
4,1514779200,2018-01-01 04:00:00 +0000 UTC,0,Dublin,53.349805,-6.26031,5.04,-4.91,4.11,6.22,...,240,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03n


#### Select required features

In [4]:
df_weather = df_weather[['dt', 'feels_like', 'wind_speed', 'weather_id',
                         'temp', 'temp_min', 'temp_max', 'humidity', 'weather_main', 'weather_description']] 

In [5]:
df_weather['feels_like'] = pd.to_numeric(df_weather['feels_like'].round(1), errors='coerce')
df_weather['wind_speed'] = pd.to_numeric(df_weather['wind_speed'].round(1), errors='coerce')
df_weather['weather_id'] = df_weather['weather_id'].astype('str')
df_weather['temp'] = pd.to_numeric(df_weather['temp'].round(1), errors='coerce')
df_weather['temp_min'] = pd.to_numeric(df_weather['temp_min'].round(1), errors='coerce')
df_weather['temp_max'] = pd.to_numeric(df_weather['temp_max'].round(1), errors='coerce')
df_weather['humidity'] = pd.to_numeric(df_weather['humidity'],downcast='integer', errors='coerce')
df_weather['weather_main'] = df_weather['weather_main'].astype('str')
df_weather['weather_description'] = df_weather['weather_description'].astype('str')

In [6]:
print("Shape of dataframe",df_weather.shape)
df_weather.head()

Shape of dataframe (8760, 10)


,dt,feels_like,wind_speed,weather_id,temp,temp_min,temp_max,humidity,weather_main,weather_description
0,1514764800,-6.5,12.9,520,4.2,3.8,5.8,87,Rain,light intensity shower rain
1,1514768400,-5.8,11.8,520,4.1,3.6,5.9,87,Rain,light intensity shower rain
2,1514772000,-5.8,12.4,802,4.6,3.8,6.0,81,Clouds,scattered clouds
3,1514775600,-5.7,12.4,802,4.6,4.0,6.1,81,Clouds,scattered clouds
4,1514779200,-4.9,11.8,802,5.0,4.1,6.2,81,Clouds,scattered clouds


### Datetime parameters

In [7]:
df_weather['dt'] = pd.to_datetime(df_weather['dt'],unit = 's')
df_weather['DAYOFSERVICE'] = df_weather['dt'].dt.date
df_weather['hour'] = df_weather['dt'].dt.hour
df_weather.drop(['dt'], axis=1, inplace= True)
df_weather = df_weather[['DAYOFSERVICE', 'hour', 'feels_like', 'wind_speed', 'weather_id',
                        'temp', 'temp_min', 'temp_max', 'humidity', 'weather_main', 'weather_description']]

In [8]:
print("Shape of dataframe",df_weather.shape)
df_weather.head()

Shape of dataframe (8760, 11)


,DAYOFSERVICE,hour,feels_like,wind_speed,weather_id,temp,temp_min,temp_max,humidity,weather_main,weather_description
0,2018-01-01,0,-6.5,12.9,520,4.2,3.8,5.8,87,Rain,light intensity shower rain
1,2018-01-01,1,-5.8,11.8,520,4.1,3.6,5.9,87,Rain,light intensity shower rain
2,2018-01-01,2,-5.8,12.4,802,4.6,3.8,6.0,81,Clouds,scattered clouds
3,2018-01-01,3,-5.7,12.4,802,4.6,4.0,6.1,81,Clouds,scattered clouds
4,2018-01-01,4,-4.9,11.8,802,5.0,4.1,6.2,81,Clouds,scattered clouds


#### UNIQUE ENTRIES

In [9]:
df_weather.nunique()

DAYOFSERVICE           365
hour                    24
feels_like             381
wind_speed              61
weather_id              23
temp                   286
temp_min               242
temp_max               241
humidity                64
weather_main             8
weather_description     24
dtype: int64

### Exteded time weather data
**Maximum value for operated extended hours is 25; i.e. 2 hour into next date.**<br>
**We are employing logic to generate weather data for upto 3 hours of extended time**
<br>

#### Extract first 3 entries for each day
$first\ 3\ entries\ for\ next\ day\ = 3\ Extended\ time\ entries\ for\ present\ day$

In [10]:
df_extendedData_weather = df_weather[(df_weather[['hour']].isin(['0','1','2'])).all(1)]

In [11]:
print("Shape of dataframe",df_extendedData_weather.shape)
df_extendedData_weather.head()

Shape of dataframe (1095, 11)


,DAYOFSERVICE,hour,feels_like,wind_speed,weather_id,temp,temp_min,temp_max,humidity,weather_main,weather_description
0,2018-01-01,0,-6.5,12.9,520,4.2,3.8,5.8,87,Rain,light intensity shower rain
1,2018-01-01,1,-5.8,11.8,520,4.1,3.6,5.9,87,Rain,light intensity shower rain
2,2018-01-01,2,-5.8,12.4,802,4.6,3.8,6.0,81,Clouds,scattered clouds
24,2018-01-02,0,-2.3,5.7,802,3.7,3.0,5.8,75,Clouds,scattered clouds
25,2018-01-02,1,-2.3,6.2,803,4.0,2.4,5.6,75,Clouds,broken clouds


#### Replace DAYOFSERVICE with previous date AND convert hour into enteded hours

In [12]:
# DAYOFSERVICE to previous date
df_extendedData_weather['DAYOFSERVICE'] = df_extendedData_weather['DAYOFSERVICE'] - pd.to_timedelta(1,unit='d')
# hours into extended hours
df_extendedData_weather['hour'] = df_extendedData_weather['hour'].add(24)

C:\Users\rasik\Anaconda3\envs\envDA\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\rasik\Anaconda3\envs\envDA\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
print("Shape of dataframe",df_extendedData_weather.shape)
df_extendedData_weather.head()

Shape of dataframe (1095, 11)


,DAYOFSERVICE,hour,feels_like,wind_speed,weather_id,temp,temp_min,temp_max,humidity,weather_main,weather_description
0,2017-12-31,24,-6.5,12.9,520,4.2,3.8,5.8,87,Rain,light intensity shower rain
1,2017-12-31,25,-5.8,11.8,520,4.1,3.6,5.9,87,Rain,light intensity shower rain
2,2017-12-31,26,-5.8,12.4,802,4.6,3.8,6.0,81,Clouds,scattered clouds
24,2018-01-01,24,-2.3,5.7,802,3.7,3.0,5.8,75,Clouds,scattered clouds
25,2018-01-01,25,-2.3,6.2,803,4.0,2.4,5.6,75,Clouds,broken clouds


**For last entry of database [In this case 31st december 2018]; extended weather data can not be obtained as entry for 1st January 2019 is not available**<br>
**Hence, we create artificial entry by duplicating last entry in database 3 times and replacing hour field with extended hour values**
<br>

#### Extended hours for last entry of database

In [14]:
df_LAST_ENTRY_weather = df_weather.loc[(df_weather.DAYOFSERVICE == df_weather.DAYOFSERVICE.max()) &
               (df_weather.hour == df_weather.hour.max())].copy()

In [15]:
df_LAST_ENTRY_weather

,DAYOFSERVICE,hour,feels_like,wind_speed,weather_id,temp,temp_min,temp_max,humidity,weather_main,weather_description
8759,2018-12-31,23,4.4,5.1,803,8.9,8.1,9.4,81,Clouds,broken clouds


In [16]:
### Add 3 artificial entries for extended time  of last entry in weather database
for _ in range(3):
    df = df_LAST_ENTRY_weather.tail(1).copy()
    df['hour'] = df['hour'] + 1
    df_LAST_ENTRY_weather = df_LAST_ENTRY_weather.append(df).sort_values('hour')

In [17]:
print("Shape of dataframe",df_LAST_ENTRY_weather.shape)
df_LAST_ENTRY_weather.head()

Shape of dataframe (4, 11)


,DAYOFSERVICE,hour,feels_like,wind_speed,weather_id,temp,temp_min,temp_max,humidity,weather_main,weather_description
8759,2018-12-31,23,4.4,5.1,803,8.9,8.1,9.4,81,Clouds,broken clouds
8759,2018-12-31,24,4.4,5.1,803,8.9,8.1,9.4,81,Clouds,broken clouds
8759,2018-12-31,25,4.4,5.1,803,8.9,8.1,9.4,81,Clouds,broken clouds
8759,2018-12-31,26,4.4,5.1,803,8.9,8.1,9.4,81,Clouds,broken clouds


### Generate EXTEDNED WEATHER dataframe 

In [18]:
df_weather_extendedTime = df_weather.append([df_extendedData_weather,df_LAST_ENTRY_weather])\
                                            .sort_values(['DAYOFSERVICE','hour']).drop_duplicates()
df_weather_extendedTime['DAYOFSERVICE'] = df_weather_extendedTime['DAYOFSERVICE'].astype('str')

In [19]:
print("Shape of dataframe",df_weather_extendedTime.shape)
df_weather_extendedTime.head()

Shape of dataframe (9858, 11)


,DAYOFSERVICE,hour,feels_like,wind_speed,weather_id,temp,temp_min,temp_max,humidity,weather_main,weather_description
0,2017-12-31,24,-6.5,12.9,520,4.2,3.8,5.8,87,Rain,light intensity shower rain
1,2017-12-31,25,-5.8,11.8,520,4.1,3.6,5.9,87,Rain,light intensity shower rain
2,2017-12-31,26,-5.8,12.4,802,4.6,3.8,6.0,81,Clouds,scattered clouds
0,2018-01-01,0,-6.5,12.9,520,4.2,3.8,5.8,87,Rain,light intensity shower rain
1,2018-01-01,1,-5.8,11.8,520,4.1,3.6,5.9,87,Rain,light intensity shower rain


#### Generate csv

In [20]:
df_weather_extendedTime.to_csv("../DB/ML/weather_extendedTime_2018.csv",index=False)

In [21]:
df_weather_extendedTime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9858 entries, 0 to 8759
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DAYOFSERVICE         9858 non-null   object 
 1   hour                 9858 non-null   int64  
 2   feels_like           9858 non-null   float64
 3   wind_speed           9858 non-null   float64
 4   weather_id           9858 non-null   object 
 5   temp                 9858 non-null   float64
 6   temp_min             9858 non-null   float64
 7   temp_max             9858 non-null   float64
 8   humidity             9858 non-null   int8   
 9   weather_main         9858 non-null   object 
 10  weather_description  9858 non-null   object 
dtypes: float64(5), int64(1), int8(1), object(4)
memory usage: 856.8+ KB
